In [ ]:
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.


In [3]:
# Descargamos uno de los modelos para español.
!python -m spacy download es_core_news_lg

2023-10-19 19:04:04.043831: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [ ]:
# Comprobamos la compatibilidad entre la versión de SpaCy de de los modelos descargados.
!python -m spacy validate

2023-10-18 08:38:38.784092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 08:38:39.816765: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.2) =================
ℹ spaCy installation: /usr/local/lib/python3.10/dist-packages/spacy

NAME              SPACY            VERSION                            
es_core_news_lg   >=3.7.0,<3.8.0   3.7.0   ✔
en_core_web_sm    >=3.6.0,<3.7.0   3.6.0   --> 3.7.0     


============================== Install updates ==============================
Use the following commands to update the packages:
python -m spacy download en_core_web_sm



In [1]:
# Importamos el módulo de SpaCy.
import spacy

In [4]:
# Cargamos uno de los modelos para español, entrenado a partir de un corpus mayoritariamente de noticias.
es_nlp = spacy.load('es_core_news_lg')

In [ ]:
from nltk.tree import Tree

In [ ]:
# Creamos funciones que nos permiten mostrar los atributos morfológicos, las dependencias y el árbol de análisis de las oraciones.
# Puedes utilizarlos para ver más claramente las oraciones con las que estés trabajando.

def tok_format(tok):
    return "_".join([tok.orth_, tok.tag_, tok.dep_])


def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(tok_format(node), [to_nltk_tree(child) for child in node.children])
    else:
        return tok_format(node)

In [ ]:
# Procesamos un texto con SpaCy y mostramos los atributos y el árbol dependencial de cada una de las oraciones.

ejemplo = """Mi perro persigue a su hijo en la bicicleta pero mi perro no sabe andar en bicicleta.
Pruebo otro ejemplo de frase. A continuación, otra frase sin vuelta de carro."""
es_doc = es_nlp(ejemplo)

for sent in es_doc.sents:
  to_nltk_tree(sent.root).pretty_print()

                                                                                  persigue_VERB_RO                                                                             
                                                                                         OT                                                                                    
        _________________________________________________________________________________|______________________________________________________________________________        
       |                          |                                     |                                        sabe_VERB_conj                                         |      
       |                          |                                     |                                 _____________|________________________________                |       
       |                          |                                     |                                |            

In [ ]:
# Observa el siguiente bloque de código, en el que se codifica sintácticamente la extracción del objeto de una pregunta.

head_word = "null"
question = "¿En qué idiomas habla tu sobrino?"

es_doc = es_nlp(question)
for sent in es_doc.sents:
  for token in sent:
    print(token, token.pos_, token.dep_, token.head)
    if (token.dep_ == "obj" or token.dep_ == "obl") and (token.pos_ == "NOUN"):
      head_word = token.text
  print('\n' + question + "\nSe pregunta por: " + head_word)

¿ PUNCT punct habla
En ADP case idiomas
qué DET det idiomas
idiomas NOUN obl habla
habla VERB ROOT habla
tu DET det sobrino
sobrino NOUN obj habla
? PUNCT punct habla

¿En qué idiomas habla tu sobrino?
Se pregunta por: sobrino


In [ ]:
# Observa ahora la aplicación de la estrategia anterior a un listado de oraciones:

sentences = """¿Qué plato quieres para cenar?
¿Cuál es tu libro favorito?
¿Qué música te gusta?
¿A qué playa vas?
¿De qué alumno es este examen?""".split('\n')

for sentence in sentences:
  head_word = "null"
  question = sentence

  sent = es_nlp(question)
  for token in sent:
    print(token, token.pos_, token.dep_, token.head)
    if (token.dep_ == "obj" or token.dep_ == "obl") and (token.pos_ == "NOUN"):
      head_word = token.text
  print('\n' + question + "\nSe pregunta por: " + head_word + '\n\n')

¿ PUNCT punct quieres
Qué DET det plato
plato NOUN obj quieres
quieres VERB ROOT quieres
para ADP mark cenar
cenar VERB advcl quieres
? PUNCT punct quieres

¿Qué plato quieres para cenar?
Se pregunta por: plato


¿ PUNCT punct libro
Cuál PRON nsubj libro
es AUX cop libro
tu DET det libro
libro NOUN ROOT libro
favorito ADJ amod libro
? PUNCT punct libro

¿Cuál es tu libro favorito?
Se pregunta por: null


¿ PUNCT punct gusta
Qué DET det música
música NOUN obj gusta
te PRON obj gusta
gusta VERB ROOT gusta
? PUNCT punct gusta

¿Qué música te gusta?
Se pregunta por: música


¿ PUNCT punct vas
A ADP case playa
qué DET det playa
playa NOUN obj vas
vas VERB ROOT vas
? PUNCT punct vas

¿A qué playa vas?
Se pregunta por: playa


¿ PUNCT punct examen
De ADP case alumno
qué DET det alumno
alumno NOUN nsubj examen
es AUX cop examen
este DET det examen
examen NOUN ROOT examen
? PUNCT punct examen

¿De qué alumno es este examen?
Se pregunta por: null




In [ ]:
# Pregunta 1: Centra tu análisis en aquellas oraciones para las que el bloque de código anterior no extrae el objeto de la pregunta.
# Categoriza los errores en categorías y refina la estrategia de extracción para que funciones también con esas oraciones.
# Comprueba, además, que continúan analizándose correctamente las oraciones del listado original.

# he hecho el códido de dos maneras distintas:
# la primera tiene un elif por cada error que quiero solucionar:

import spacy

sentences = """¿Qué plato quieres para cenar?
¿Cuál es tu libro favorito?
¿Qué música te gusta?
¿A qué playa vas?
¿De qué alumno es este examen?""".split('\n')

for sentence in sentences:
  head_word = "null"
  question = sentence

  sent = es_nlp(question)
  for token in sent:
    print(token, token.pos_, token.dep_, token.head)
    if (token.dep_ == "obj" or token.dep_ == "obl") and (token.pos_ == "NOUN"):
      head_word = token.text
      break
    if (token.dep_ == "nsubj") and (token.pos_ == "NOUN"):
      head_word = token.text
      break
    if (token.dep_ == "ROOT") and (token.pos_ == "NOUN"):
      head_word = token.text
      break
  print('\n' + question + "\nSe pregunta por: " + head_word + '\n\n')

¿ PUNCT punct quieres
Qué DET det plato
plato NOUN obj quieres

¿Qué plato quieres para cenar?
Se pregunta por: plato


¿ PUNCT punct libro
Cuál PRON nsubj libro
es AUX cop libro
tu DET det libro
libro NOUN ROOT libro

¿Cuál es tu libro favorito?
Se pregunta por: libro


¿ PUNCT punct gusta
Qué DET det música
música NOUN obj gusta

¿Qué música te gusta?
Se pregunta por: música


¿ PUNCT punct vas
A ADP case playa
qué DET det playa
playa NOUN obj vas

¿A qué playa vas?
Se pregunta por: playa


¿ PUNCT punct examen
De ADP case alumno
qué DET det alumno
alumno NOUN nsubj examen

¿De qué alumno es este examen?
Se pregunta por: alumno




In [ ]:
# Pregunta 2: Comprueba, con algunos ejemplo más, que tu estrategia analiza correctamente otras oraciones de las mismas categorías que has descrito.

sentences = """¿Te gustan los gatos?
¿Cuál es tu deporte favorito?
¿Qué carrera has estudiado?
¿A qué ciudad vas?
¿De qué niño es este juguete?
¿Cuánto cuesta aquí el pantalón?""".split('\n')

for sentence in sentences:
  head_word = "null"
  question = sentence

  sent = es_nlp(question)
  for token in sent:
    print(token, token.pos_, token.dep_, token.head)
    if (token.dep_ == "obj" or token.dep_ == "obl") and (token.pos_ == "NOUN"):
      head_word = token.text
      break
    if (token.dep_ == "nsubj") and (token.pos_ == "NOUN"):
      head_word = token.text
      break
    if (token.dep_ == "ROOT") and (token.pos_ == "NOUN"):
      head_word = token.text
      break
  print('\n' + question + "\nSe pregunta por: " + head_word + '\n\n')

¿ PUNCT punct gustan
Te PRON obj gustan
gustan VERB ROOT gustan
los DET det gatos
gatos NOUN nsubj gustan

¿Te gustan los gatos?
Se pregunta por: gatos


¿ PUNCT punct deporte
Cuál PRON nsubj deporte
es AUX cop deporte
tu DET det deporte
deporte NOUN ROOT deporte

¿Cuál es tu deporte favorito?
Se pregunta por: deporte


¿ PUNCT punct estudiado
Qué DET det carrera
carrera NOUN obj estudiado

¿Qué carrera has estudiado?
Se pregunta por: carrera


¿ PUNCT punct vas
A ADP case ciudad
qué DET det ciudad
ciudad NOUN obl vas

¿A qué ciudad vas?
Se pregunta por: ciudad


¿ PUNCT punct juguete
De ADP case qué
qué PRON det niño
niño NOUN nsubj juguete

¿De qué niño es este juguete?
Se pregunta por: niño


¿ PUNCT punct cuesta
Cuánto PRON obj cuesta
cuesta VERB ROOT cuesta
aquí ADV advmod cuesta
el DET det pantalón
pantalón NOUN nsubj cuesta

¿Cuánto cuesta aquí el pantalón
Se pregunta por: pantalón




In [ ]:
# Pregunta 3: Comprueba si tu estrategia falla en alguna categoría de oraciones más y reflexiona sobre qué estrategia podría seguirse para contemplar dicha casuística.
# Es posible que la salida de este modelo de SpaCy no permite solucionar el problema. Si es así, comenta igualmente qué información lingüística sería necesario codificar para solucionar el problema.
sentences = """¿Quién de tus amigos fue el que te invitó?
¿Cuáles son tus deportes favoritos?
¿Quién vino?
¿Cuántas veces has visto la película """.split('\n')

for sentence in sentences:
  head_word = "null"
  question = sentence

  sent = es_nlp(question)
  for token in sent:
    print(token, token.pos_, token.dep_, token.head)
    if (token.dep_ == "obj" or token.dep_ == "obl") and (token.pos_ == "NOUN"):
      head_word = token.text
      break
    if (token.dep_ == "nsubj") and (token.pos_ == "NOUN"):
      head_word = token.text
      break
    if (token.dep_ == "ROOT") and (token.pos_ == "NOUN"):
      head_word = token.text
      break
  print('\n' + question + "\nSe pregunta por: " + head_word + '\n\n')

¿ PUNCT punct fue
Quién PRON nsubj fue
de ADP case amigos
tus DET det amigos
amigos NOUN nmod Quién
fue AUX ROOT fue
el DET det invitó
que PRON obj invitó
te PRON iobj invitó
invitó VERB ccomp fue
? PUNCT punct fue

¿Quién de tus amigos fue el que te invitó?
Se pregunta por: null


¿ PUNCT punct deportes
Cuáles PRON nsubj deportes
son AUX cop deportes
tus DET det deportes
deportes NOUN ROOT deportes

¿Cuáles son tus deportes favoritos?
Se pregunta por: deportes


¿ PUNCT punct vino
Quién PRON nsubj vino
vino VERB ROOT vino
? PUNCT punct vino

¿Quién vino?
Se pregunta por: null


¿ PUNCT punct visto
Cuántas DET det veces
veces NOUN obl visto

¿Cuántas veces has visto la película 
Se pregunta por: veces




In [ ]:
# Me da error en la primera y en la cuarta frase
# La manera de solucionarlo es añadir como posibilidad los pronombres de sujeto y obl.
# Sin embargo necesitaríamos una gramática que infiriera que cuando se pregunta "¿Cómo te llamas?" no
# se pregunta por cómo, sino por su nombre.
sentences = """¿Quién de tus amigos fue el que te invitó?
¿Cuáles son tus deportes favoritos?
¿Cómo te llamas?
¿Cuántas veces has visto la película """.split('\n')

for sentence in sentences:
  head_word = "null"
  question = sentence

  sent = es_nlp(question)
  for token in sent:
    print(token, token.pos_, token.dep_, token.head)
    if (token.dep_ == "obj" or token.dep_ == "obl") and (token.pos_ == "NOUN" or token.pos_ == "PRON"):
      head_word = token.text
      break
    if (token.dep_ == "nsubj") and (token.pos_ == "NOUN" or token.pos_ == "PRON"):
      head_word = token.text
      break
    if (token.dep_ == "ROOT") and (token.pos_ == "NOUN"):
      head_word = token.text
      break
  print('\n' + question + "\nSe pregunta por: " + head_word + '\n\n')

¿ PUNCT punct fue
Quién PRON nsubj fue

¿Quién de tus amigos fue el que te invitó?
Se pregunta por: Quién


¿ PUNCT punct deportes
Cuáles PRON nsubj deportes

¿Cuáles son tus deportes favoritos?
Se pregunta por: Cuáles


¿ PUNCT punct llamas
Cómo PRON obl llamas

¿Cómo te llamas?
Se pregunta por: Cómo


¿ PUNCT punct visto
Cuántas DET det veces
veces NOUN obl visto

¿Cuántas veces has visto la película 
Se pregunta por: veces




In [ ]:
# Observa el siguiente bloque de código, con el que se pretende extraer tripletas (tuplas de 3 elementos) sujeto-verbo-objeto

oraciones = "Juan compra caramelos. Juan se queja sobre los exámenes"
es_doc = es_nlp(oraciones)
for sent in es_doc.sents:
  t_1 = "null"
  t_2 = "null"
  t_3 = "null"
  for token in sent:
      print(token, token.pos_, token.dep_, token.head)
      if token.dep_ == "nsubj":
          t_1 = token.text
      if token.dep_ == "ROOT":
          t_2 = token.text
      if token.dep_ == "obj":
          t_3 = token.text
  print('\n' + sent.orth_ + "\n(" + t_1 + ', ' + t_2 + ', ' + t_3 + ")\n")

Juan PROPN nsubj compra
compra VERB ROOT compra
caramelos NOUN obj compra
. PUNCT punct compra

Juan compra caramelos.
(Juan, compra, caramelos)

Juan PROPN nsubj queja
se PRON expl:pv queja
queja VERB ROOT queja
sobre ADP case exámenes
los DET det exámenes
exámenes NOUN obl queja

Juan se queja sobre los exámenes
(Juan, queja, null)



In [5]:
# Pregunta 4: Refina el bloque de código anterior para que extraiga objetos con preposición.
# Añade más oraciones para comprobar la cobertura de tu estrategia. En el caso de encontrar errores, categorízalos y reflexiona sobre qué estrategia podría seguirse para contemplar dicha casuística.
oraciones = "Juan compra caramelos. Juan se queja sobre los exámenes.Juan trabaja con su padre.Juan habla de la vida.Juan se duerme encima del sofá.Juan sale con sus amigos. A Juan le encanta pasear por la playa"

es_doc = es_nlp(oraciones)
for sent in es_doc.sents:
  t_1 = "null"
  t_2 = "null"
  t_3 = "null"
  for token in sent:
      print(token, token.pos_, token.dep_, token.head)
      if(token.dep_ == "nsubj") or (token.pos_ == "PROPN"):
          t_1 = token.text
      if token.dep_ == "ROOT":
          t_2 = token.text
      if (token.dep_ == "obl" or token.dep_ == "obj" ) and (token.pos_ == "NOUN"):
          t_3 = token.text
  print('\n' + sent.orth_ + "\n(" + t_1 + ', ' + t_2 + ', ' + t_3 + ")\n")

Juan PROPN nsubj compra
compra VERB ROOT compra
caramelos NOUN obj compra
. PUNCT punct compra

Juan compra caramelos.
(Juan, compra, caramelos)

Juan PROPN nsubj queja
se PRON expl:pv queja
queja VERB ROOT queja
sobre ADP case exámenes
los DET det exámenes
exámenes NOUN obl queja
. PUNCT punct queja

Juan se queja sobre los exámenes.
(Juan, queja, exámenes)

Juan PROPN nsubj trabaja
trabaja VERB ROOT trabaja
con ADP case padre
su DET det padre
padre NOUN obl trabaja
. PUNCT punct trabaja

Juan trabaja con su padre.
(Juan, trabaja, padre)

Juan PROPN nsubj habla
habla VERB ROOT habla
de ADP case vida
la DET det vida
vida NOUN obj habla
. PUNCT punct habla

Juan habla de la vida.
(Juan, habla, vida)

Juan PROPN nsubj duerme
se PRON expl:pv duerme
duerme VERB ROOT duerme
encima ADV advmod duerme
del ADP case sofá
sofá NOUN obl duerme
. PUNCT punct duerme

Juan se duerme encima del sofá.
(Juan, duerme, sofá)

Juan PROPN nsubj sale
sale VERB ROOT sale
con ADP case amigos
sus DET det amigos

In [ ]:
# Pregunta 5: Diseña una estrategia para extraer listas de tuplas nombre-complemento(adyacente) de un texto, tal y como se muestra a continuación:
# Pista: una posible solución del problema pasa por utilizar un id de cada sustantivo del texto.
# Para ello, podemos utilizar la posición del token en el texto, accesible a partir del atributo i del objeto Token.

texto = """El peor día de mi vida fue cuando descubrí los enormes problemas de sobrevivir en una ciudad extranjera sin red de apoyo.
Cuando crees que la cosa no puede ser peor, te das cuenta de que la vida siempre puede sorprenderte con un nuevo imprevisto.
"""
es_doc = es_nlp(texto)
tuplas = []
for sent in es_doc.sents:
  to_nltk_tree(sent.root).pretty_print()

for sent in es_doc.sents:
    for token in sent:
        if (token.pos_ == "NOUN"):
            for child in token.children:
                if (child.pos_ == "ADJ") or (child.dep_ == "nmod") or (child.pos_ == "VERB"):
                    tuplas.append((token.text, child.text))

print(tuplas)


                                                                                              fue_AUX_ROOT                                                                                                                          
                  _________________________________________________________________________________|_________________________________________________________________________________________________________________________        
                 |                                                                          descubrí_VERB_cc                                                                                                                 |      
                 |                                                                                omp                                                                                                                        |      
                 |                                                     ____________

In [ ]:
# Observa la salida esperada del ejercicio y fíjate en la inclusión errónea de la tupla (ciudad, red).
# ¿A qué obedece ese fallo? ¿Debemos refinar nuestra estrategia para solventarlo?


[(día, peor), (día, vida), (problemas, enormes), (problemas, sobrevivir), (ciudad, extranjera), (ciudad, red), (red, apoyo), (imprevisto, nuevo)]


In [ ]:
# se debe a que "red" cumple los criterios de sustantivo y nmod, si le decimos que no tenga en cuenta las palabras
# que cumplen esos criterios, entonces eliminaría también (día, vida). El problema es que el árbol lo identifica
#como un modificador de "ciudad" cuando realmente no lo es.
